In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import json
from distutils.dir_util import copy_tree
import shutil

In [23]:
!pip install python==3.70

ERROR: Could not find a version that satisfies the requirement python==3.70 (from versions: none)
ERROR: No matching distribution found for python==3.70


In [15]:
#Uninstall new JAX
!pip uninstall jax jaxlib -y
#GPU frontend
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
#CPU frontend
#!pip install jax[cpu]==0.3.10
#Downgrade Pytorch
!pip uninstall torch torchvision -y
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2


Found existing installation: jax 0.3.14
Uninstalling jax-0.3.14:
  Successfully uninstalled jax-0.3.14
Found existing installation: jaxlib 0.3.8+cuda11.cudnn82
Uninstalling jaxlib-0.3.8+cuda11.cudnn82:
  Successfully uninstalled jaxlib-0.3.8+cuda11.cudnn82
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.7/939.7 kB 64.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 MB 11.9 MB/s eta 0:00:0000:0100:01
  Created wheel for jax: filename=jax-0.3.10-py3-none-any.whl size=1088051 sha256=497af6444c7fdaaf3c8aaf6f940ed5f81e0b244ef83b6cdc1fa867dd76b08a59
  Stored in directory: /root/.cache/pip/wheels/14/4a/ff/e9ddfa09012c67d22f926a7873c546c04e722969e8d86f84ec
Successfully built jax
Found existing installation: torch 1.12.0+cu116
Uninstalling torch-1.12.0+cu116:
  Successfully uninstalled torch-1.12.0+cu116
Found existing installation: torchvision 

In [17]:
pip install setuptools==59.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 65.6 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 63.1.0
    Uninstalling setuptools-63.1.0:
      Successfully uninstalled setuptools-63.1.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
path = "/datasets/data/"
datasets=[path+"axial",path+"coronal",path+"saggital"]
labels=[path+"train-abnormal",path+"train-acl",path+"train-meniscus"]

In [ ]:
if(not os.path.exists("Datasets")):
    os.mkdir("Datasets")

for l in labels:
    try:
        l_name=l.split("-")[1]
        os.mkdir("Datasets/"+l_name)

    except:
        print("Directory already exists")
    try:
        os.mkdir("Datasets/"+l_name+"/0/")
    except:
        pass
    try:
        os.mkdir("Datasets/"+l_name+"/1/")
    except:
        pass

In [ ]:
data_images={}
for d in datasets:
    data_images[d]=np.load(d+".npy")

In [ ]:
label_data={}
for l in labels:
    l_dat=pd.read_csv(l+".csv",names=['img','label'])
    label_data[l]=l_dat

In [6]:
filler = np.zeros([256,256,3],dtype=np.uint8)
for i in range(data_images[path+'axial'].shape[0]):
    ax=data_images[path+'axial'][i]
    cor=data_images[path+'coronal'][i]
    sag=data_images[path+'saggital'][i]
    first_row=np.concatenate((ax,cor),axis=1)
    second_row=np.concatenate((sag,filler),axis=1)
    out_image=np.concatenate((first_row,second_row),axis=0)
    im=Image.fromarray(out_image)
    for l in labels:
        if(label_data[l]['label'].iloc[i]==1):
            im.save("Datasets/"+l.split("-")[1]+"/1/"+str(i)+".png")
        else:
            im.save("Datasets/"+l.split("-")[1]+"/0/"+str(i)+".png")
            
    print(i,label_data[l].iloc[i]['label']==1,"done!")          
    

0 False done!
1 True done!
2 False done!
3 True done!
4 False done!
5 True done!
6 False done!
7 False done!
8 False done!
9 False done!
10 True done!
11 False done!
12 False done!
13 False done!
14 False done!
15 False done!
16 True done!
17 False done!
18 True done!
19 True done!
20 True done!
21 False done!
22 True done!
23 False done!
24 False done!
25 False done!
26 False done!
27 True done!
28 True done!
29 False done!
30 True done!
31 False done!
32 True done!
33 True done!
34 False done!
35 True done!
36 True done!
37 True done!
38 True done!
39 True done!
40 False done!
41 False done!
42 True done!
43 True done!
44 False done!
45 True done!
46 False done!
47 False done!
48 False done!
49 True done!
50 False done!
51 False done!
52 True done!
53 False done!
54 True done!
55 False done!
56 False done!
57 True done!
58 False done!
59 True done!
60 False done!
61 False done!
62 False done!
63 True done!
64 False done!
65 False done!
66 False done!
67 True done!
68 False done!
69 F

In [7]:
for l in labels:
    l_name=l.split("-")[1]
    path="Datasets/"+l_name
    zeros=os.listdir(path+"/0")
    ones=os.listdir(path+"/1")
    zeros_class=[(l_name+"/0/"+x,0) for x in zeros]
    ones_class=[(l_name+"/1/"+x,1) for x in ones]
    out_class=zeros_class+ones_class
    out_class.sort(key=lambda x:float(x[0].split("/")[-1].split(".")[0]))
    out_class={"labels":out_class}
    json.dump(out_class,open(path+"/dataset.json","w"))
    pass

# Balance the dataset!

In [20]:
#if path already exists, remove it before copying with copytree()
if os.path.exists("Datasets_balanced"):
    shutil.rmtree("Datasets_balanced")

shutil.copytree("/notebooks/Datasets", "/notebooks/Datasets_balanced")


'/notebooks/Datasets_balanced'

In [21]:
for l in labels:
    path="/notebooks/Datasets_balanced/"+l.split("-")[1]
    zeros=len(os.listdir(path+"/0"))
    ones=len(os.listdir(path+"/1"))
    np.random.seed(0) #seed for making sure everytime it runs delete the same images, you can change that
    if(zeros>ones):
        zero_dirs=os.listdir(path+"/0")
        to_be_deleted=np.random.choice(zero_dirs,zeros-ones,replace=False)
        for d in to_be_deleted:
            os.remove(path+"/0/"+d)
    elif(ones>zeros):
        one_dirs=os.listdir(path+"/1")
        to_be_deleted=np.random.choice(one_dirs,ones-zeros,replace=False)
        for d in to_be_deleted:
            os.remove(path+"/1/"+d)
    else:
        print("No need to delete")
        
for l in labels:
    l_name=l.split("-")[1]
    path="/notebooks/Datasets_balanced/"+"/"+l_name
    zeros=os.listdir(path+"/0")
    ones=os.listdir(path+"/1")
    zeros_class=[("0/"+x,0) for x in zeros]
    ones_class=[("1/"+x,1) for x in ones]
    out_class=zeros_class+ones_class
    out_class.sort(key=lambda x:float(x[0].split("/")[-1].split(".")[0]))
    out_class={"labels":out_class}
    json.dump(out_class,open(path+"/dataset.json","w"))
    pass

In [22]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git stylegan
%cd stylegan
#!pip install ninia

Cloning into 'stylegan'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 3.80 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/notebooks/stylegan/stylegan


In [24]:
for l in labels:
    l_name=l.split("-")[1]
    !python dataset_tool.py --source /notebooks/Datasets_balanced/{l_name} --dest data/{l_name}

/notebooks/stylegan/stylegan/dataset_tool.py:205: DeprecationWarning: BOX is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BOX instead.
  resample = { 'box': PIL.Image.BOX, 'lanczos': PIL.Image.LANCZOS }[resize_filter]
/notebooks/stylegan/stylegan/dataset_tool.py:205: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resample = { 'box': PIL.Image.BOX, 'lanczos': PIL.Image.LANCZOS }[resize_filter]
100%|█████████████████████████████████████████| 434/434 [00:53<00:00,  8.07it/s]
/notebooks/stylegan/stylegan/dataset_tool.py:205: DeprecationWarning: BOX is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BOX instead.
  resample = { 'box': PIL.Image.BOX, 'lanczos': PIL.Image.LANCZOS }[resize_filter]
/notebooks/stylegan/stylegan/dataset_tool.py:205: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  

In [ ]:
#for l in labels:
    #l_name=l.split("-")[1]
!python train.py --data /notebooks/stylegan/data/acl --cond 1 --kimg 1000 --outdir outputs --gpus 2 --metrics None


Training options:
{
  "num_gpus": 2,
  "image_snapshot_ticks": 50,
  "network_snapshot_ticks": 50,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/notebooks/stylegan/data/acl",
    "use_labels": true,
    "max_size": 416,
    "xflip": false,
    "resolution": 512
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512,
    "num_fp16_re

In [12]:
import os, shutil
shutil.rmtree("/notebooks/stylegan/data")